In [ ]:
## Data collection
import nltk 
nltk.download('gutenberg')
from nltk.corpus import gutenberg 
import pandas as pd


## Load the dataset
data = gutenberg.raw('shakespeare-hamlet.txt')

## Save to a file
with open('hamlet.txt', 'w') as file:
    file.write(data)
    

In [3]:
## Data preprocessing
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

## Load the dataset and apply padding sequences and tokenizer

with open('hamlet.txt', 'r') as file:
    text = file.read().lower()

## Tokenize the text

tokenizer = Tokenizer()  ## Creating indexes for the words
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
total_words

4818

In [5]:
## Creating input sequences

input_sequences =[]
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [7]:
## Apply pad sequences
max_sequence_len=max([len(x) for x in input_sequences])
max_sequence_len

14

In [ ]:
## Converting into an array
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
input_sequences

In [12]:
## Create predictors and label

import tensorflow as tf
x,y =input_sequences[:,:-1], input_sequences[:,-1]

In [ ]:
## Creating categorical features
y = tf.keras.utils.to_categorical(y, num_classes=total_words)
y

In [18]:
## Splitting the dataset into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

In [ ]:
## Train the LSTM RNN

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

## Define the model
model = Sequential()
model.add(Embedding(total_words,100))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation="softmax"))

## Build the model
model.build(input_shape=(None, max_sequence_len))

## Compile the model 
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
## Training the model 
history = model.fit(x_train, y_train, epochs=50, validation_data=(x_test, y_test), verbose=1)


In [30]:
## Importing EarlyStopping callback to monitor validation loss
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience =5, restore_best_weights = True)

In [ ]:
## Training the model again for 100 epochs
history = model.fit(x_train, y_train, epochs=100, validation_data=(x_test, y_test), verbose=1)


In [33]:
## Function to predict the next word
def predict_next(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]  ## Ensure sequence length matches max_sequence_len -1
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None 

In [ ]:
## Trying an input to see how the model performs in predicting next word
input_text = "To be or not to be"
print(f"Input Text: {input_text} ")
max_sequence_len=model.input_shape[1]+1
next_word = predict_next(model, tokenizer, input_text, max_sequence_len)
print(f"Next Word Prediction: {next_word}")

Input Text: To be or not to be 
Next Word Prediction: nothing


In [ ]:
## Saving the model
model.save("Next_Word_LSTM.h5")

## Saving the tokenizer as a .pkl file

import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)  ## Highest protocol minimizes file size and speeds up serialization

In [41]:
## Trying another input 

input_text = "To offer it the shew of"  ## input taken from dataset, line: 196
print(f"Input Text: {input_text} ")
max_sequence_len=model.input_shape[1]+1
next_word = predict_next(model, tokenizer, input_text, max_sequence_len)
print(f"Next Word Prediction: {next_word}")

Input Text: To offer it the shew of 
Next Word Prediction: violence


In [42]:
## Trying another input 
input_text = "For it is as the Ayre,"  ## input taken from dataset, line: 197
print(f"Input Text: {input_text} ")
max_sequence_len=model.input_shape[1]+1
next_word = predict_next(model, tokenizer, input_text, max_sequence_len)
print(f"Next Word Prediction: {next_word}")

Input Text: For it is as the Ayre, 
Next Word Prediction: invulnerable
